### library

In [91]:
!pip install catboost

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [93]:
import catboost
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

In [94]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import random
import torch

np.random.seed(92)
random.seed(92)
torch.manual_seed(92)

%matplotlib inline

df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('submission.csv')
df_train

,bant_submit,expected_budget,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,lead_date,customer_type,customer_history,enterprise,...,transfer_agreement,ver_cus,ver_pro,ver_win_rate_x,ver_win_rate_mean_upper,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,0.75,NaN,/DKI Jakarta/Indonesia,AS,NaN,33931,2022-10-15,End-Customer,Existing,Enterprise,...,Y,0,0,0.000060,1.0,0.128571,hospital & health care,General Hospital,870,False
1,0.25,NaN,/Minneola/United States,ID,0.390244,42490,2023-05-31,NaN,New,SMB,...,N,0,0,0.000572,0.0,0.048630,education,NaN,501,True
2,0.25,NaN,4880 Stevens Creek Blvd Cloud IT/Hollywood/Un...,IT,NaN,10350,2023-04-19,Solution Eco-Partner,New,SMB,...,Y,0,0,NaN,NaN,NaN,NaN,NaN,375,True
3,1.00,"$100,000 ~ $500,000",/Wiesbaden/Germany,ID,0.017544,19750,2021-10-30,NaN,New,SMB,...,Y,0,0,0.000572,0.0,0.048630,education,NaN,359,False
4,0.50,NaN,//Singapore,ID,NaN,45802,2023-04-11,Specifier/ Influencer,New,SMB,...,Y,0,0,NaN,NaN,NaN,NaN,NaN,198,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59294,0.75,NaN,/Noida/India,ID,NaN,17289,2021-07-02,NaN,Existing,Enterprise,...,Y,0,0,NaN,NaN,NaN,NaN,NaN,174,False
59295,0.75,NaN,/Sao Bernardo do Campo/Brazil,ID,NaN,23686,2021-07-27,NaN,New,SMB,...,Y,0,0,NaN,NaN,NaN,NaN,NaN,211,False
59296,0.50,NaN,/Alexandria/Egypt,AS,0.049689,29370,2021-08-26,NaN,Existing,Enterprise,...,Y,0,0,0.000298,0.0,0.020121,residential (home),NaN,56,False
59297,0.50,NaN,/Mogi guaçu SP /Brazil,AS,NaN,22609,2022-04-25,NaN,New,Enterprise,...,Y,0,0,NaN,NaN,NaN,NaN,NaN,96,False


In [95]:
df_train.describe()

,bant_submit,com_reg_ver_win_rate,customer_idx,historical_existing_cnt,id_strategic_ver,it_strategic_ver,idit_strategic_ver,lead_desc_length,prefer_ver_count,prefer_ver_mean,ver_cus,ver_pro,ver_win_rate_x,ver_win_rate_mean_upper,ver_win_ratio_per_bu,lead_owner
count,59299.000000,14568.000000,59299.000000,13756.000000,3444.0,1121.0,4565.0,59299.000000,18417.000000,18417.000000,59299.000000,59299.000000,18417.000000,18417.000000,15304.000000,59299.000000
mean,0.634593,0.091685,27114.556333,19.912184,1.0,1.0,1.0,79.271590,0.152536,0.094945,0.041603,0.050810,0.001117,0.207526,0.049288,262.563888
std,0.286066,0.150988,14653.911888,44.697938,0.0,0.0,0.0,132.551067,0.112766,0.131403,0.199681,0.219612,0.001104,0.405546,0.027949,231.605681
min,0.000000,0.003788,2.000000,0.000000,1.0,1.0,1.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000002,0.000000,0.011583,0.000000
25%,0.500000,0.019900,14913.000000,1.000000,1.0,1.0,1.0,7.000000,0.058091,0.019648,0.000000,0.000000,0.000298,0.000000,0.026846,97.000000
50%,0.500000,0.049180,26774.000000,4.000000,1.0,1.0,1.0,29.000000,0.145228,0.046220,0.000000,0.000000,0.000572,0.000000,0.049840,175.000000
75%,1.000000,0.074949,40368.500000,19.000000,1.0,1.0,1.0,92.000000,0.211618,0.103511,0.000000,0.000000,0.001183,0.000000,0.064566,374.000000
max,1.000000,1.000000,47466.000000,1394.000000,1.0,1.0,1.0,1264.000000,1.000000,1.000000,1.000000,1.000000,0.003079,1.000000,0.285714,1114.000000


In [96]:
df_train.isnull().sum()

bant_submit                    0
expected_budget            35177
customer_country             982
business_unit                  0
com_reg_ver_win_rate       44731
customer_idx                   0
lead_date                      0
customer_type              43961
customer_history               0
enterprise                     0
historical_existing_cnt    45543
id_strategic_ver           55855
it_strategic_ver           58178
idit_strategic_ver         54734
customer_job               18733
lead_from_channel           3295
lead_description           30203
lead_desc_length               0
event_name                  1132
inquiry_type                 941
prefer_ver_count           40882
prefer_ver_mean            40882
product_category           19374
product_subcategory        50064
product_modelname          50070
customer_country.1           982
customer_position              0
response_corporate             0
expected_timeline          30863
transfer_agreement           530
ver_cus   

In [97]:
def unique_element(data, column):
    print(f'{column}열에는 {data[column].nunique()}개의 요소가 있습니다.')
    
for column in df_train.columns.tolist():
    unique_element(df_train, column)

bant_submit열에는 5개의 요소가 있습니다.
expected_budget열에는 95개의 요소가 있습니다.
customer_country열에는 15399개의 요소가 있습니다.
business_unit열에는 5개의 요소가 있습니다.
com_reg_ver_win_rate열에는 80개의 요소가 있습니다.
customer_idx열에는 35112개의 요소가 있습니다.
lead_date열에는 794개의 요소가 있습니다.
customer_type열에는 33개의 요소가 있습니다.
customer_history열에는 2개의 요소가 있습니다.
enterprise열에는 2개의 요소가 있습니다.
historical_existing_cnt열에는 136개의 요소가 있습니다.
id_strategic_ver열에는 1개의 요소가 있습니다.
it_strategic_ver열에는 1개의 요소가 있습니다.
idit_strategic_ver열에는 1개의 요소가 있습니다.
customer_job열에는 560개의 요소가 있습니다.
lead_from_channel열에는 62개의 요소가 있습니다.
lead_description열에는 20084개의 요소가 있습니다.
lead_desc_length열에는 909개의 요소가 있습니다.
event_name열에는 1132개의 요소가 있습니다.
inquiry_type열에는 71개의 요소가 있습니다.
prefer_ver_count열에는 23개의 요소가 있습니다.
prefer_ver_mean열에는 29개의 요소가 있습니다.
product_category열에는 357개의 요소가 있습니다.
product_subcategory열에는 330개의 요소가 있습니다.
product_modelname열에는 665개의 요소가 있습니다.
customer_country.1열에는 15399개의 요소가 있습니다.
customer_position열에는 117개의 요소가 있습니다.
response_corporate열에는 53개의 요소가 있습니다.
expected_timeline열에는 449개의

### 전처리 레츠고

In [98]:
# business unit

def convert_unit(x):
    if x == 'CM':
        return 'ID'
    elif x == 'Solution':
        return 'AS'
    else:
        return x
df_train['business_unit'] = df_train['business_unit'].apply(lambda x : convert_unit(x))
df_test['business_unit'] = df_test['business_unit'].apply(lambda x : convert_unit(x))

In [99]:
# 중복값 삭제

df_train = df_train.drop_duplicates().reset_index(drop = True)
df_train

,bant_submit,expected_budget,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,lead_date,customer_type,customer_history,enterprise,...,transfer_agreement,ver_cus,ver_pro,ver_win_rate_x,ver_win_rate_mean_upper,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,0.75,NaN,/DKI Jakarta/Indonesia,AS,NaN,33931,2022-10-15,End-Customer,Existing,Enterprise,...,Y,0,0,0.000060,1.0,0.128571,hospital & health care,General Hospital,870,False
1,0.25,NaN,/Minneola/United States,ID,0.390244,42490,2023-05-31,NaN,New,SMB,...,N,0,0,0.000572,0.0,0.048630,education,NaN,501,True
2,0.25,NaN,4880 Stevens Creek Blvd Cloud IT/Hollywood/Un...,IT,NaN,10350,2023-04-19,Solution Eco-Partner,New,SMB,...,Y,0,0,NaN,NaN,NaN,NaN,NaN,375,True
3,1.00,"$100,000 ~ $500,000",/Wiesbaden/Germany,ID,0.017544,19750,2021-10-30,NaN,New,SMB,...,Y,0,0,0.000572,0.0,0.048630,education,NaN,359,False
4,0.50,NaN,//Singapore,ID,NaN,45802,2023-04-11,Specifier/ Influencer,New,SMB,...,Y,0,0,NaN,NaN,NaN,NaN,NaN,198,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57060,0.75,NaN,/Noida/India,ID,NaN,17289,2021-07-02,NaN,Existing,Enterprise,...,Y,0,0,NaN,NaN,NaN,NaN,NaN,174,False
57061,0.75,NaN,/Sao Bernardo do Campo/Brazil,ID,NaN,23686,2021-07-27,NaN,New,SMB,...,Y,0,0,NaN,NaN,NaN,NaN,NaN,211,False
57062,0.50,NaN,/Alexandria/Egypt,AS,0.049689,29370,2021-08-26,NaN,Existing,Enterprise,...,Y,0,0,0.000298,0.0,0.020121,residential (home),NaN,56,False
57063,0.50,NaN,/Mogi guaçu SP /Brazil,AS,NaN,22609,2022-04-25,NaN,New,Enterprise,...,Y,0,0,NaN,NaN,NaN,NaN,NaN,96,False


In [100]:
# expected_timeline
## is_converted 비율

# _, - 제거
df_train['expected_timeline'] = df_train['expected_timeline'].str.replace('_', ' ')
df_train['expected_timeline'] = df_train['expected_timeline'].str.replace('-', '~')

df_test['expected_timeline'] = df_test['expected_timeline'].str.replace('_', ' ')
df_test['expected_timeline'] = df_test['expected_timeline'].str.replace('-', '~')

# budget 부족 => yet budget
df_train['expected_timeline'] = df_train['expected_timeline'].astype(str)
df_train['expected_timeline'].replace('nan', 'others', inplace = True)
df_train['expected_timeline'].fillna('others', inplace = True)
df_train.loc[df_train['expected_timeline'].str.contains('budget', case=False), 'expected_timeline'] = 'yet budget'

df_test['expected_timeline'] = df_test['expected_timeline'].astype(str)
df_test['expected_timeline'].replace('nan', 'others', inplace = True)
df_test['expected_timeline'].fillna('others', inplace = True)
df_test.loc[df_test['expected_timeline'].str.contains('budget', case=False), 'expected_timeline'] = 'yet budget'

# less than 5 months
df_train.loc[df_train['expected_timeline'] == 'less than 5 months', 'expected_timeline'] = '3 months ~ 6 months'
df_train.loc[df_train['expected_timeline'] == 'one month', 'expected_timeline'] = 'less than 3 months'
df_train.loc[df_train['expected_timeline'] == 'less then 6 months', 'expected_timeline'] = '3 months ~ 6 months'
df_train.loc[df_train['expected_timeline'] == 'more then 3 months', 'expected_timeline'] = 'less than 3 months'
df_train.loc[df_train['expected_timeline'] == '45 days', 'expected_timeline'] = 'less than 3 months'
df_train.loc[df_train['expected_timeline'] == '4/8 months', 'expected_timeline'] = '3 months ~ 6 months'
df_train.loc[df_train['expected_timeline'] == 'less than 6 months', 'expected_timeline'] = '3 months ~ 6 months'

df_test.loc[df_test['expected_timeline'] == 'less than 5 months', 'expected_timeline'] = '3 months ~ 6 months'
df_test.loc[df_test['expected_timeline'] == 'one month', 'expected_timeline'] = 'less than 3 months'
df_test.loc[df_test['expected_timeline'] == 'less then 6 months', 'expected_timeline'] = '3 months ~ 6 months'
df_test.loc[df_test['expected_timeline'] == 'more then 3 months', 'expected_timeline'] = 'less than 3 months'
df_test.loc[df_test['expected_timeline'] == '45 days', 'expected_timeline'] = 'less than 3 months'
df_test.loc[df_test['expected_timeline'] == '4/8 months', 'expected_timeline'] = '3 months ~ 6 months'
df_test.loc[df_test['expected_timeline'] == 'less than 6 months', 'expected_timeline'] = '3 months ~ 6 months'

# 온점 제거
df_train['expected_timeline'] = df_train['expected_timeline'].str.replace('.', '')
df_test['expected_timeline'] = df_test['expected_timeline'].str.replace('.', '')

# call later
df_train.loc[df_train['expected_timeline'].str.contains('call later', case=False), 'expected_timeline'] = 'call later'
df_train.loc[df_train['expected_timeline'].str.contains('call back later', case=False), 'expected_timeline'] = 'call later'
df_train.loc[df_train['expected_timeline'].str.contains('call back', case=False), 'expected_timeline'] = 'call later'

df_train.loc[df_train['expected_timeline'].str.contains('2022', case=False), 'expected_timeline'] = 'less than 3 months'
df_train.loc[df_train['expected_timeline'].str.contains('less than 3 months', case=False), 'expected_timeline'] = 'less than 3 months'
df_train.loc[df_train['expected_timeline'].str.contains('3 months', case=False), 'expected_timeline'] = 'less than 3 months'

df_test.loc[df_test['expected_timeline'].str.contains('call later', case=False), 'expected_timeline'] = 'call later'
df_test.loc[df_test['expected_timeline'].str.contains('call back later', case=False), 'expected_timeline'] = 'call later'
df_test.loc[df_test['expected_timeline'].str.contains('call back', case=False), 'expected_timeline'] = 'call later'

df_test.loc[df_test['expected_timeline'].str.contains('2022', case=False), 'expected_timeline'] = 'less than 3 months'
df_test.loc[df_test['expected_timeline'].str.contains('less than 3 months', case=False), 'expected_timeline'] = 'less than 3 months'
df_test.loc[df_test['expected_timeline'].str.contains('3 months', case=False), 'expected_timeline'] = 'less than 3 months'


# requirement , interested, require 포함
df_train.loc[df_train['expected_timeline'].str.contains('requirement', case=False), 'expected_timeline'] = 'not requirements'
df_train.loc[df_train['expected_timeline'].str.contains('interested', case=False), 'expected_timeline'] = 'not requirements'
df_train.loc[df_train['expected_timeline'].str.contains('require', case=False), 'expected_timeline'] = 'not requirements'
df_train.loc[df_train['expected_timeline'].str.contains('not', case=False), 'expected_timeline'] = 'not requirements'
df_train.loc[df_train['expected_timeline'].str.contains('no', case=False), 'expected_timeline'] = 'not requirements'
df_train.loc[df_train['expected_timeline'].str.contains("couldn't", case=False), 'expected_timeline'] = 'not requirements'
df_train.loc[df_train['expected_timeline'].str.contains("didn't", case=False), 'expected_timeline'] = 'not requirements'

df_test.loc[df_test['expected_timeline'].str.contains('requirement', case=False), 'expected_timeline'] = 'not requirements'
df_test.loc[df_test['expected_timeline'].str.contains('interested', case=False), 'expected_timeline'] = 'not requirements'
df_test.loc[df_test['expected_timeline'].str.contains('require', case=False), 'expected_timeline'] = 'not requirements'
df_test.loc[df_test['expected_timeline'].str.contains('not', case=False), 'expected_timeline'] = 'not requirements'
df_test.loc[df_test['expected_timeline'].str.contains('no', case=False), 'expected_timeline'] = 'not requirements'
df_test.loc[df_test['expected_timeline'].str.contains("couldn't", case=False), 'expected_timeline'] = 'not requirements'
df_test.loc[df_test['expected_timeline'].str.contains("didn't", case=False), 'expected_timeline'] = 'not requirements'

# detail, discuss 포함
df_train.loc[df_train['expected_timeline'].str.contains('detail', case=False), 'expected_timeline'] = 'discuss later'
df_train.loc[df_train['expected_timeline'].str.contains('discuss', case=False), 'expected_timeline'] = 'discuss later'

df_test.loc[df_test['expected_timeline'].str.contains('detail', case=False), 'expected_timeline'] = 'discuss later'
df_test.loc[df_test['expected_timeline'].str.contains('discuss', case=False), 'expected_timeline'] = 'discuss later'

# demo, quote
df_train.loc[df_train['expected_timeline'].str.contains('demo', case=False), 'expected_timeline'] = 'send the quote'
df_train.loc[df_train['expected_timeline'].str.contains('quote', case=False), 'expected_timeline'] = 'send the quote'

df_test.loc[df_test['expected_timeline'].str.contains('demo', case=False), 'expected_timeline'] = 'send the quote'
df_test.loc[df_test['expected_timeline'].str.contains('quote', case=False), 'expected_timeline'] = 'send the quote'

In [101]:
filter_ = ['less than 3 months', '3 months ~ 6 months', 'more than a year', '9 months ~ 1 year', '6 months ~ 9 months', 'others']
df_train = df_train[df_train['expected_timeline'].isin(filter_)].reset_index(drop = True)
df_train

,bant_submit,expected_budget,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,lead_date,customer_type,customer_history,enterprise,...,transfer_agreement,ver_cus,ver_pro,ver_win_rate_x,ver_win_rate_mean_upper,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,0.75,NaN,/DKI Jakarta/Indonesia,AS,NaN,33931,2022-10-15,End-Customer,Existing,Enterprise,...,Y,0,0,0.000060,1.0,0.128571,hospital & health care,General Hospital,870,False
1,0.25,NaN,/Minneola/United States,ID,0.390244,42490,2023-05-31,NaN,New,SMB,...,N,0,0,0.000572,0.0,0.048630,education,NaN,501,True
2,0.25,NaN,4880 Stevens Creek Blvd Cloud IT/Hollywood/Un...,IT,NaN,10350,2023-04-19,Solution Eco-Partner,New,SMB,...,Y,0,0,NaN,NaN,NaN,NaN,NaN,375,True
3,1.00,"$100,000 ~ $500,000",/Wiesbaden/Germany,ID,0.017544,19750,2021-10-30,NaN,New,SMB,...,Y,0,0,0.000572,0.0,0.048630,education,NaN,359,False
4,0.50,NaN,//Singapore,ID,NaN,45802,2023-04-11,Specifier/ Influencer,New,SMB,...,Y,0,0,NaN,NaN,NaN,NaN,NaN,198,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56357,0.75,NaN,/Noida/India,ID,NaN,17289,2021-07-02,NaN,Existing,Enterprise,...,Y,0,0,NaN,NaN,NaN,NaN,NaN,174,False
56358,0.75,NaN,/Sao Bernardo do Campo/Brazil,ID,NaN,23686,2021-07-27,NaN,New,SMB,...,Y,0,0,NaN,NaN,NaN,NaN,NaN,211,False
56359,0.50,NaN,/Alexandria/Egypt,AS,0.049689,29370,2021-08-26,NaN,Existing,Enterprise,...,Y,0,0,0.000298,0.0,0.020121,residential (home),NaN,56,False
56360,0.50,NaN,/Mogi guaçu SP /Brazil,AS,NaN,22609,2022-04-25,NaN,New,Enterprise,...,Y,0,0,NaN,NaN,NaN,NaN,NaN,96,False


In [102]:
# 문자열 -> 소문자로

convert_to_lower = ['customer_country', 'business_unit', 'customer_type', 'enterprise', 'customer_job', 'inquiry_type',
                   'product_category','product_subcategory', 'product_modelname', 'customer_position', 'response_corporate',
                   'expected_timeline', 'business_area', 'business_subarea', 'expected_budget', 'event_name']

for convert_column in convert_to_lower:
    df_train[convert_column] = df_train[convert_column].apply(lambda x : str(x).lower())
    df_test[convert_column] = df_test[convert_column].apply(lambda x : str(x).lower())

In [103]:
# expected_budget 시작

In [104]:
# '_'-> 띄어쓰기로 교체
df_train['expected_budget'] = df_train['expected_budget'].str.replace('_', ' ')
df_test['expected_budget'] = df_test['expected_budget'].str.replace('_', ' ')

In [105]:
# 문자 대체

# 띄어쓰기 두 번 -> 한 번
df_train['expected_budget'] = df_train['expected_budget'].str.replace('  ', ' ')
df_test['expected_budget'] = df_test['expected_budget'].str.replace('  ', ' ')

# '-' -> '~'
df_train['expected_budget'] = df_train['expected_budget'].str.replace('-', '~')
df_test['expected_budget'] = df_test['expected_budget'].str.replace('-', '~')

# 유로 -> 달러 (환율 고려하여 1달러 == 1유로)
df_train['expected_budget'] = df_train['expected_budget'].str.replace('€', '$')
df_test['expected_budget'] = df_test['expected_budget'].str.replace('€', '$')

# 파운드 -> 달러 (환율 고려하여 1달러 == 1파운드) -> 1.2정도
df_train['expected_budget'] = df_train['expected_budget'].str.replace('£', '$')
df_test['expected_budget'] = df_test['expected_budget'].str.replace('£', '$')

# '.' -> ','
df_train['expected_budget'] = df_train['expected_budget'].str.replace('.', ',')
df_test['expected_budget'] = df_test['expected_budget'].str.replace('.', ',')

# 'to less than' -> '~'
df_train['expected_budget'] = df_train['expected_budget'].str.replace('to less than', '~')
df_test['expected_budget'] = df_test['expected_budget'].str.replace('to less than', '~')

# on~going -> ongoing
df_train['expected_budget'] = df_train['expected_budget'].str.replace('on~going', 'ongoing')
df_test['expected_budget'] = df_test['expected_budget'].str.replace('on~going', 'ongoing')

# 10,000 $ ~ 50,000 $ -> $10,000 ~ $50,000
df_train['expected_budget'] = df_train['expected_budget'].str.replace('10,000 $ ~ 50,000 $', '$10,000 ~ $50,000')
df_test['expected_budget'] = df_test['expected_budget'].str.replace('10,000 $ ~ 50,000 $', '$10,000 ~ $50,000')

# 500,000$ ~ 1,000,000$ -> $500,000 ~ $1,000,000
df_train['expected_budget'] = df_train['expected_budget'].str.replace('500,000$ ~ 1,000,000$', '$500,000 ~ $1,000,000')
df_test['expected_budget'] = df_test['expected_budget'].str.replace('500,000$ ~ 1,000,000$', '$500,000 ~ $1,000,000')

In [106]:
# 영어 외 언어

# menos de(~이내- 스페인어) -> less than
df_train['expected_budget'] = df_train['expected_budget'].str.replace('menos de', 'less than')
df_test['expected_budget'] = df_test['expected_budget'].str.replace('menos de', 'less than')

# weniger als(~이내 -독일어) -> less than
df_train['expected_budget'] = df_train['expected_budget'].str.replace('weniger als', 'less than')
df_test['expected_budget'] = df_test['expected_budget'].str.replace('weniger als', 'less than')

# über(~이상 - 독일어) -> more than
df_train['expected_budget'] = df_train['expected_budget'].str.replace('über', 'less than')
df_test['expected_budget'] = df_test['expected_budget'].str.replace('über', 'less than')

# entre 500,000$ y 1,000,000$(~~부터 ~~까지 - 스페인) -> $500,000 ~ 1,000,000
df_train['expected_budget'] = df_train['expected_budget'].str.replace('entre 500,000$ y 1,000,000$', '$500,000 ~ $1,000,000')
df_test['expected_budget'] = df_test['expected_budget'].str.replace('entre 500,000$ y 1,000,000$', '$500,000 ~ $1,000,000')

# entre $50,000 y 100,000$ -> $50,000 ~ $100,000
df_train['expected_budget'] = df_train['expected_budget'].str.replace('entre $50,000 y 100,000$', '$50,000 ~ $100,000')
df_test['expected_budget'] = df_test['expected_budget'].str.replace('entre $50,000 y 100,000$', '$50,000 ~ $100,000')

# entre $10,000 y $50,000
df_train['expected_budget'] = df_train['expected_budget'].str.replace('entre $10,000 y $50,000', '$10,000 ~ $50,000')
df_test['expected_budget'] = df_test['expected_budget'].str.replace('entre $10,000 y $50,000', '$10,000 ~ $50,000')

In [107]:
# 결측치, 모른다고 기재

# 'n/a;n/a' -> 'don\'t know/not sure'
df_train['expected_budget'] = df_train['expected_budget'].str.replace('n/a;n/a', 'don\'t know/not sure')
df_test['expected_budget'] = df_test['expected_budget'].str.replace('n/a;n/a', 'don\'t know/not sure')

# 'nan' -> 'don\'t know/not sure'
df_train['expected_budget'] = df_train['expected_budget'].str.replace('nan', 'don\'t know/not sure')
df_test['expected_budget'] = df_test['expected_budget'].str.replace('nan', 'don\'t know/not sure')

# no know -> don'tknow/not sure
df_train['expected_budget'] = df_train['expected_budget'].str.replace('no know', 'don\'t know/not sure')
df_test['expected_budget'] = df_test['expected_budget'].str.replace('no know', 'don\t know/not sure')

In [108]:
# 달러 기호 위치

# 10,000$ -> $10,000
df_train['expected_budget'] = df_train['expected_budget'].str.replace('10,000$', '$10,000')
df_test['expected_budget'] = df_test['expected_budget'].str.replace('10,000$', '$10,000')

# 50,000$ -> $50,000
df_train['expected_budget'] = df_train['expected_budget'].str.replace('50,000$', '$50,000')
df_test['expected_budget'] = df_test['expected_budget'].str.replace('50,000$', '$50,000')

In [109]:
# '1m' ->'1,000,000'
df_train['expected_budget'] = df_train['expected_budget'].str.replace('1m', '1,000,000')
df_test['expected_budget'] = df_test['expected_budget'].str.replace('1m', '1,000,000')

# '10m' -> '10,000,000'
df_train['expected_budget'] = df_train['expected_budget'].str.replace('10m', '10,000,000')
df_test['expected_budget'] = df_test['expected_budget'].str.replace('10m', '10,000,000')

# '25m' -> 25,000,000
df_train['expected_budget'] = df_train['expected_budget'].str.replace('25m', '25,000,000')
df_test['expected_budget'] = df_test['expected_budget'].str.replace('25m', '25,000,000')

# '10,000,000 ~ 20,000,000' -> $10,000,000 ~ $25,000,000
df_train['expected_budget'] = df_train['expected_budget'].str.replace('10,000,000 ~ 20,000,000', '$10,000,000 ~ $25,000,000')
df_test['expected_budget'] = df_test['expected_budget'].str.replace('10,000,000 ~ 20,000,000', '$10,000,000 ~ $25,000,000')

# R$ : 브라질 헤알, 


# COP: 콜롬비아 페소, 1 COP = 0.00027 USD

In [110]:
# 세미콜론 ';' 포함

# ;$500,000 ~ $1,000,000 -> ''
df_train['expected_budget'] = df_train['expected_budget'].str.replace(';$500,000 ~ $1,000,000', '')
df_test['expected_budget'] = df_test['expected_budget'].str.replace(';$500,000 ~ $1,000,000', '')

# ;$100,000 ~ $500,000 -> ''
df_train['expected_budget'] = df_train['expected_budget'].str.replace(';$100,000 ~ $500,000', '')
df_test['expected_budget'] = df_test['expected_budget'].str.replace(';$100,000 ~ $500,000', '')

# ;$50,000 ~ $100,000 -> ''
df_train['expected_budget'] = df_train['expected_budget'].str.replace(';$50,000 ~ $100,000', '')
df_test['expected_budget'] = df_test['expected_budget'].str.replace(';$50,000 ~ $100,000', '')

# ;less than $10,000-> ''
df_train['expected_budget'] = df_train['expected_budget'].str.replace(';less than $10,000', '')
df_test['expected_budget'] = df_test['expected_budget'].str.replace(';less than $10,000', '')

# ;more than $1,000,000-> ''
df_train['expected_budget'] = df_train['expected_budget'].str.replace(';more than $1,000,000', '')
df_test['expected_budget'] = df_test['expected_budget'].str.replace(';more than $1,000,000', '')

# ;$10,000 ~ $50,000 -> ''
df_train['expected_budget'] = df_train['expected_budget'].str.replace(';$10,000 ~ $50,000', '')
df_test['expected_budget'] = df_test['expected_budget'].str.replace(';$10,000 ~ $50,000', '')

In [111]:
df_train['expected_budget'].value_counts()

expected_budget
don't know/not sure            33157
less than $100,000              6488
less than $10,000               5706
$10,000 ~ $50,000               3081
$100,000 ~ $500,000             2104
more than $1,000,000            1803
$50,000 ~ $100,000              1292
less than $50,000                748
$500,000 ~ $1,000,000            687
ongoing                          539
$100,000 ~ $200,000              178
$200,000 ~ $400,000              152
more than $50,000                108
less than $7,000                  80
$400,000 ~ $600,000               65
$600,000 ~ $800,000               59
$800,000 ~ $1,000,000             53
$1,000,000 ~ $10,000,000           9
$7,000 ~ $35,000                   6
entre $10,000 y $50,000            6
more than $700,000                 5
less than $1,000,000               4
$10,000,000 ~ $25,000,000          3
$35,000 ~ $70,000                  3
entre $50,000 y 100,000$           3
less than $123,000                 1
more than $100,000    

In [112]:
"""# 123 ->100
df_train['expected_budget'] = df_train['expected_budget'].str.replace('123', '100')
df_test['expected_budget'] = df_test['expected_budget'].str.replace('123', '100')

# budget -> ongoing
df_train['expected_budget'] = df_train['expected_budget'].str.replace('budget', 'ongoing')
df_test['expected_budget'] = df_test['expected_budget'].str.replace('budget', 'ongoing')


# less than $7,000 -> less than $10,000
df_train['expected_budget'] = df_train['expected_budget'].str.replace('less than $7,000', 'less than $10,000')
df_test['expected_budget'] = df_test['expected_budget'].str.replace('less than $7,000', 'less than $10,000')

# '$7,000 ~ $35,000'-> '$10,000 ~ $50,000'
df_train['expected_budget'] = df_train['expected_budget'].str.replace('$7,000 ~ $35,000', '$10,000 ~ $50,000')
df_test['expected_budget'] = df_test['expected_budget'].str.replace('$7,000 ~ $35,000', '$10,000 ~ $50,000')

# '30,000 $ ~ 50,000 $' -> '$10,000 ~ $50,000'
df_train['expected_budget'] = df_train['expected_budget'].str.replace('30,000 $ ~ 50,000 $', '$10,000 ~ $50,000')
df_test['expected_budget'] = df_test['expected_budget'].str.replace('30,000 $ ~ 50,000 $', '$10,000 ~ $50,000')

# 'less than $50,000' -> '$10,000 ~ $50,000'
df_train['expected_budget'] = df_train['expected_budget'].str.replace('less than $50,000', '$10,000 ~ $50,000')
df_test['expected_budget'] = df_test['expected_budget'].str.replace('less than $50,000', '$10,000 ~ $50,000')"""

"# 123 ->100\ndf_train['expected_budget'] = df_train['expected_budget'].str.replace('123', '100')\ndf_test['expected_budget'] = df_test['expected_budget'].str.replace('123', '100')\n\n# budget -> ongoing\ndf_train['expected_budget'] = df_train['expected_budget'].str.replace('budget', 'ongoing')\ndf_test['expected_budget'] = df_test['expected_budget'].str.replace('budget', 'ongoing')\n\n\n# less than $7,000 -> less than $10,000\ndf_train['expected_budget'] = df_train['expected_budget'].str.replace('less than $7,000', 'less than $10,000')\ndf_test['expected_budget'] = df_test['expected_budget'].str.replace('less than $7,000', 'less than $10,000')\n\n# '$7,000 ~ $35,000'-> '$10,000 ~ $50,000'\ndf_train['expected_budget'] = df_train['expected_budget'].str.replace('$7,000 ~ $35,000', '$10,000 ~ $50,000')\ndf_test['expected_budget'] = df_test['expected_budget'].str.replace('$7,000 ~ $35,000', '$10,000 ~ $50,000')\n\n# '30,000 $ ~ 50,000 $' -> '$10,000 ~ $50,000'\ndf_train['expected_budget'] 

In [113]:
"""filter_ = ["don't know/not sure", '$100,000 ~ $500,000', '$50,000 ~ $100,000', 'more than $1,000,000', '$200,000 ~ $400,000', 
           'less than $10,000', 'less than $100,000', '$10,000 ~ $50,000', 'ongoing', '$500,000 ~ $1,000,000', 'more than $50,000', 
           '$400,000 ~ $600,000', '$100,000 ~ $200,000', '$600,000 ~ $800,000', '$800,000 ~ $1,000,000', '$10,000,000 ~ $25,000,000',
           'less than $1,000,000']
df_train = df_train[df_train['expected_budget'].isin(filter_)].reset_index(drop = True)
df_train"""

'filter_ = ["don\'t know/not sure", \'$100,000 ~ $500,000\', \'$50,000 ~ $100,000\', \'more than $1,000,000\', \'$200,000 ~ $400,000\', \n           \'less than $10,000\', \'less than $100,000\', \'$10,000 ~ $50,000\', \'ongoing\', \'$500,000 ~ $1,000,000\', \'more than $50,000\', \n           \'$400,000 ~ $600,000\', \'$100,000 ~ $200,000\', \'$600,000 ~ $800,000\', \'$800,000 ~ $1,000,000\', \'$10,000,000 ~ $25,000,000\',\n           \'less than $1,000,000\']\ndf_train = df_train[df_train[\'expected_budget\'].isin(filter_)].reset_index(drop = True)\ndf_train'

In [114]:
# country

df_train['country'] = df_train.customer_country.str.split('/').str[-1].str.strip() 
df_test['country'] = df_test.customer_country.str.split('/').str[-1].str.strip() 

In [115]:
df_train.loc[((df_train['country'] == '') & (df_train['customer_country'] == '//')), 'country'] = 'Unknown'
df_test.loc[((df_test['country'] == '') & (df_test['customer_country'] == '//')), 'country'] = 'Unknown'

df_train.loc[((df_train['country'] == '') & (df_train['customer_country'] != '//')), 'country'] ='united states'
df_test.loc[((df_test['country'] == '') & (df_test['customer_country'] != '//')), 'country'] = 'united states'

In [116]:
country_series = df_train['country'].value_counts()
convert_usa_list = country_series[country_series <= 2].index.tolist()
convert_usa_list.append('700 patroon creek blvdalbanyny12206')
convert_usa_list.append('us')

df_train['country'] = df_train['country'].apply(lambda x : 'united states' if x in convert_usa_list else x)
df_test['country'] = df_test['country'].apply(lambda x : 'united states' if x in convert_usa_list else x)

In [117]:
# customer type

def convert_type(x):
    if x in ['end-customer', 'commercial end-user', 'end-user']:
        return 'end customer'
    elif x in ['other', 'etc.']:
        return 'others'
    elif x in ['distributor', 'reseller']:
        return 'dealer/distributor'
    elif x == 'homeowner':
        return 'home owner'
    elif x == 'specifier/ influencer':
        return 'specifier / influencer'
    elif x == 'hvac engineer':
        return 'engineer'
    elif x == 'administrator':
        return 'manager / director'
    elif x == 'technical assistant':
        return 'technician'
    elif x== 'installer':
        return 'installer/contractor'
    elif x == 'consultant':
        return 'architect/consultant'
    elif x == 'software/solution provider':
        return 'software / solution provider'
    elif x == 'interior designer':
        return 'architect/consultant'
    elif x == 'system integrator':
        return 'developer'
    else:
        return x
    
df_train['customer_type'] = df_train['customer_type'].apply(lambda x : convert_type(x))
df_test['customer_type'] = df_test['customer_type'].apply(lambda x : convert_type(x))

In [118]:
# customer_type

def convert_customer(x) :
        try :
            x = x.replace('-', '')
            x = x.replace(' ', '')
        except : pass
        return x

df_train['customer_type'] = df_train['customer_type'].apply(lambda x : convert_customer(x))
df_test['customer_type'] = df_test['customer_type'].apply(lambda x : convert_customer(x))

In [119]:
# inquiry_type

def convert_inquiry(x) :
        try :
            x = x.replace('_', '')
        except : pass
        return x

df_train['inquiry_type'] = df_train['inquiry_type'].apply(lambda x : convert_inquiry(x))
df_test['inquiry_type'] = df_test['inquiry_type'].apply(lambda x : convert_inquiry(x))

In [120]:
# customer job

def convert_job(x):
    if x == 'arquitecto/consultor':
        return 'consulting'
    elif x == 'other':
        return 'others'
    elif x == 'purchasers':
        return 'purchasing'
    else:
        return x
    
df_train['customer_job'] = df_train['customer_job'].apply(lambda x : convert_job(x))
df_test['customer_job'] = df_test['customer_job'].apply(lambda x : convert_job(x))

In [121]:
# customer position

def convert_position(x):
    if x == 'commercial end-user':
        return 'end-user'
    elif x == 'vicepresident':
        return 'vice president'
    elif x == 'asst prof.':
        return 'assistant professor'
    elif x == 'other' :
        return 'others'
    else:
        return x
    
df_train['customer_position'] = df_train['customer_position'].apply(lambda x : convert_position(x))
df_test['customer_position'] = df_test['customer_position'].apply(lambda x : convert_position(x))

In [122]:
# lead_from_channel 컬럼 - 숫자 제거
def remove_numbers(text):
    if isinstance(text, str):
        return ''.join([i for i in text if not i.isdigit()])
    else:
        return text

df_train['lead_from_channel'] = df_train['lead_from_channel'].apply(remove_numbers)
df_test['lead_from_channel'] = df_test['lead_from_channel'].apply(remove_numbers)



# lead_from_channel 전처리
# 결측치 -> etc.
df_train['lead_from_channel'].fillna('etc.', inplace = True)
df_test['lead_from_channel'].fillna('etc.', inplace = True)

df_train['lead_from_channel'] = df_train['lead_from_channel'].str.replace('_', ' ')
df_test['lead_from_channel'] = df_test['lead_from_channel'].str.replace('_', ' ')

# lg.co
df_train.loc[df_train['lead_from_channel'] == 'lg.co', 'lead_from_channel'] = 'lg.com'
df_test.loc[df_test['lead_from_channel'] == 'lg.co', 'lead_from_channel'] = 'lg.com'

# newletter
df_train.loc[df_train['lead_from_channel'].str.contains('newsletter', case=False), 'lead_from_channel'] = 'newsletter'
df_test.loc[df_test['lead_from_channel'].str.contains('newsletter', case=False), 'lead_from_channel'] = 'newsletter'

# db
df_train.loc[df_train['lead_from_channel'].str.contains('db', case=False), 'lead_from_channel'] = 'db'
df_test.loc[df_test['lead_from_channel'].str.contains('db', case=False), 'lead_from_channel'] = 'db'

# exhibition
df_train.loc[df_train['lead_from_channel'].str.contains('exhibition', case=False), 'lead_from_channel'] = 'exhibition'
df_test.loc[df_test['lead_from_channel'].str.contains('exhibition', case=False), 'lead_from_channel'] = 'exhibition'

df_train.loc[df_train['lead_from_channel'] == 'exhibiton', 'lead_from_channel'] = 'exhibition'
df_test.loc[df_test['lead_from_channel'] == 'exhibiton', 'lead_from_channel'] = 'exhibition'

# seminar
df_train.loc[df_train['lead_from_channel'].str.contains('sem', case=False), 'lead_from_channel'] = 'seminar'
df_test.loc[df_test['lead_from_channel'].str.contains('sem', case=False), 'lead_from_channel'] = 'seminar'

# webinar
df_train.loc[df_train['lead_from_channel'].str.contains('webinar', case=False), 'lead_from_channel'] = 'webinar'
df_test.loc[df_test['lead_from_channel'].str.contains('webinar', case=False), 'lead_from_channel'] = 'webinar'

# mql
df_train.loc[df_train['lead_from_channel'].str.contains('mql', case=False), 'lead_from_channel'] = 'mql'
df_test.loc[df_test['lead_from_channel'].str.contains('mql', case=False), 'lead_from_channel'] = 'mql'

# campaign, event, website -> etc
df_train.loc[df_train['lead_from_channel'].str.contains('campaign', case=False), 'lead_from_channel'] = 'etc.'
df_test.loc[df_test['lead_from_channel'].str.contains('campaign', case=False), 'lead_from_channel'] = 'etc.'

df_train.loc[df_train['lead_from_channel'].str.contains('event', case=False), 'lead_from_channel'] = 'etc.'
df_test.loc[df_test['lead_from_channel'].str.contains('event', case=False), 'lead_from_channel'] = 'etc.'

df_train.loc[df_train['lead_from_channel'].str.contains('website', case=False), 'lead_from_channel'] = 'etc.'
df_test.loc[df_test['lead_from_channel'].str.contains('website', case=False), 'lead_from_channel'] = 'etc.'

In [123]:
# 결측치 처리

lower_character = ['business_area', 'inquiry_type', 'expected_timeline', 'customer_country', 'business_unit', 'customer_type',
                  'enterprise', 'customer_job', 'product_category', 'product_subcategory', 'product_modelname',
                  'customer_position', 'business_area', 'lead_from_channel']
null_median = ['com_reg_ver_win_rate', 'historical_existing_cnt', 'ver_win_rate_x', 'ver_win_ratio_per_bu']
    
for median_column in null_median:
    df_train[median_column] = df_train[median_column].fillna(df_train[median_column].median())
    df_test[median_column] = df_test[median_column].fillna(df_train[median_column].median())

def preprocess_text(input_text):
    return re.sub(r'[^a-z0-9]', ' ', input_text.lower())
    
for char_column in lower_character:
    df_train[char_column] = df_train[char_column].apply(lambda x : preprocess_text(str(x)))
    df_test[char_column] = df_test[char_column].apply(lambda x : preprocess_text(str(x)))

In [124]:
def label_encoding(series: pd.Series) -> pd.Series:
    """범주형 데이터를 시리즈 형태로 받아 숫자형 데이터로 변환합니다."""

    my_dict = {}

    # 모든 요소를 문자열로 변환
    series = series.astype(str)

    for idx, value in enumerate(sorted(series.unique())):
        my_dict[value] = idx
    series = series.map(my_dict)

    return series

In [125]:
# 레이블 인코딩할 칼럼들
label_columns = [
    "country",
    "business_subarea",
    "business_area",
    "business_unit",
    "customer_type",
    "enterprise",
    "customer_job",
    "inquiry_type",
    "product_category",
    "product_subcategory",
    "product_modelname",
    "customer_position",
    "response_corporate",
    "expected_timeline",
    "is_converted",
    "customer_history",
    "lead_from_channel",
    "transfer_agreement",
    "expected_budget",
    "event_name",
    "lead_date"
]


df_all = pd.concat([df_train[label_columns], df_test[label_columns]])

for col in label_columns:
    df_all[col] = label_encoding(df_all[col])

In [126]:
int_to_float = ['lead_desc_length']

df_train[int_to_float] = df_train[int_to_float].astype(np.float64)
df_test[int_to_float] = df_test[int_to_float].astype(np.float64)

In [127]:
df_train.isnull().sum()

bant_submit                    0
expected_budget                0
customer_country               0
business_unit                  0
com_reg_ver_win_rate           0
customer_idx                   0
lead_date                      0
customer_type                  0
customer_history               0
enterprise                     0
historical_existing_cnt        0
id_strategic_ver           53097
it_strategic_ver           55249
idit_strategic_ver         51984
customer_job                   0
lead_from_channel              0
lead_description           28453
lead_desc_length               0
event_name                     0
inquiry_type                   0
prefer_ver_count           38800
prefer_ver_mean            38800
product_category               0
product_subcategory            0
product_modelname              0
customer_country.1           974
customer_position              0
response_corporate             0
expected_timeline              0
transfer_agreement           530
ver_cus   

In [128]:
# customer_idx 가중치 (is_converted가 1인 비율 높은 것)
df_train['wei_cus_idx'] = df_train['customer_idx'].apply(lambda x : 1 if x in [25096] else 0)
df_test['wei_cus_idx'] = df_test['customer_idx'].apply(lambda x : 1 if x in [25096] else 0)

# lead_owner 가중치 (is_converted가 1인 비율 높은 것)
df_train['wei_lead_owner'] = df_train['lead_owner'].apply(lambda x : 1 if x in [487, 437, 480] else 0)
df_test['wei_lead_owner'] = df_test['lead_owner'].apply(lambda x : 1 if x in [487, 437, 480] else 0)

In [129]:
# 변수 삭제 (bant_submit, it_strategic_ver : feature importance가 작아서 삭제)
df_train.drop(['customer_country.1', 'it_strategic_ver', 'idit_strategic_ver', 'customer_country', 'bant_submit'], axis=1, inplace=True)
df_test.drop(['customer_country.1', 'it_strategic_ver', 'idit_strategic_ver', 'customer_country', 'bant_submit'], axis=1, inplace=True)

# 결측치를 빈 문자열로 대체합니다.
df_train["lead_description"].fillna("", inplace=True)
df_test["lead_description"].fillna("", inplace=True)

# 나머지는 모두 0으로 결측치 보간
df_train.fillna(0, inplace=True)
df_test.fillna(0, inplace=True)

In [130]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import SparsePCA, TruncatedSVD, LatentDirichletAllocation, NMF

svd_ = TruncatedSVD(n_components = 10)

tfidf = TfidfVectorizer(
    min_df=0.01,  # 전체 학습 데이터 중 1% 이상 나온 단어만 사용합니다.
    ngram_range=(1, 2),
    # 또한, 단어와 두 단어를 함께 사용하는 경우를 모두 사용합니다.
)

tfidf.fit(df_train['lead_description'])

tfidf_train = tfidf.transform(df_train["lead_description"])
tfidf_test = tfidf.transform(df_test["lead_description"])

svd_train_col = svd_.fit_transform(tfidf_train)
svd_test_col = svd_.transform(tfidf_test)

In [131]:
# TF-IDF로 변환된 데이터를 원본 데이터에 결합합니다.
df_train = pd.concat(
    [
        df_train.drop(columns=["lead_description"]),
        pd.DataFrame(svd_train_col, columns=np.arange(10)),
    ],
    axis=1,
)

df_test = pd.concat(
    [
        df_test.drop(columns=["lead_description"]),
        pd.DataFrame(svd_test_col, columns = np.arange(10)),
    ],
    axis=1,
)

# 컬럼명을 str 타입으로 변환합니다.
df_train.columns = df_train.columns.astype(str)
df_test.columns = df_test.columns.astype(str)

In [132]:
df_test

,id,expected_budget,business_unit,com_reg_ver_win_rate,customer_idx,lead_date,customer_type,customer_history,enterprise,historical_existing_cnt,...,0,1,2,3,4,5,6,7,8,9
0,2547,don't know/not sure,id,0.044643,19119,2023-07-12,endcustomer,New,smb,4.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,32086,"less than $10,000",as,0.147059,35859,2023-08-05,endcustomer,New,enterprise,4.0,...,0.251493,0.009624,-0.075126,-0.104187,-0.037233,0.095518,0.027773,0.317199,-0.156316,-0.072153
2,32209,"$10,000 ~ $50,000",as,0.461538,38658,2023-08-20,endcustomer,New,smb,4.0,...,0.342355,0.045087,-0.220965,-0.136603,-0.063918,0.142668,0.039530,-0.046058,-0.127773,0.219151
3,27680,ongoing,as,0.049180,29043,2023-06-20,endcustomer,New,enterprise,4.0,...,0.051706,0.004954,-0.049751,-0.002579,-0.002984,-0.013534,-0.006416,0.008228,-0.007432,0.014432
4,35109,"less than $10,000",id,0.028926,42617,2023-06-08,channelpartner,New,smb,4.0,...,0.303776,-0.124053,0.004358,0.212029,0.028287,-0.131175,-0.033069,-0.041004,-0.136848,-0.131698
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21336,26080,don't know/not sure,as,0.049180,31080,2023-07-31,nan,New,enterprise,4.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
21337,65292,"$400,000 ~ $600,000",as,0.049180,43632,2023-06-20,nan,Existing,smb,271.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
21338,27489,ongoing,as,0.049180,39855,2023-08-30,specifier influencer,New,smb,4.0,...,0.235611,0.031091,0.135517,0.039883,-0.026080,0.191438,0.057880,0.063255,0.349068,-0.074266
21339,62108,don't know/not sure,as,0.049180,47466,2023-09-15,endcustomer,Existing,enterprise,16.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### train, test split

In [133]:
for col in label_columns:  
    df_train[col] = df_all.iloc[: len(df_train)][col]
    df_test[col] = df_all.iloc[len(df_train) :][col]

x_train = df_train.drop('is_converted', axis = 1).copy()
x_val = df_test.drop('is_converted', axis = 1).copy()
y_train = df_train['is_converted'].copy()
'''
x_train, x_val, y_train, y_val = train_test_split(
    df_train.drop("is_converted", axis=1),
    df_train["is_converted"],
    test_size=0.2,
    shuffle=True
)
'''

'\nx_train, x_val, y_train, y_val = train_test_split(\n    df_train.drop("is_converted", axis=1),\n    df_train["is_converted"],\n    test_size=0.2,\n    shuffle=True\n)\n'

In [134]:
classes = np.unique(y_train)
weights = compute_class_weight(class_weight = 'balanced', classes = classes, y = y_train)
class_weights = dict(zip(classes, weights))
class_weights

{0: 0.5436359427447046, 1: 6.22922192749779}

In [135]:
x_train

,expected_budget,business_unit,com_reg_ver_win_rate,customer_idx,lead_date,customer_type,customer_history,enterprise,historical_existing_cnt,id_strategic_ver,...,0,1,2,3,4,5,6,7,8,9
0,31,0,0.049180,33931,566,5,0,0,4.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,31,1,0.390244,42490,793,10,1,1,4.0,0.0,...,0.328107,-0.273446,0.322130,0.032074,0.006438,-0.077971,-0.019580,-0.117008,-0.084724,-0.045846
2,31,2,0.049180,10350,751,14,1,1,4.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,4,1,0.017544,19750,223,10,1,1,4.0,0.0,...,0.503778,0.023009,0.106001,-0.002878,0.029533,-0.168092,-0.054957,0.166759,-0.017095,0.012006
4,31,1,0.049180,45802,743,15,1,1,4.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56357,31,1,0.049180,17289,105,10,0,0,3.0,0.0,...,0.439812,-0.052894,-0.047225,-0.000473,-0.014082,-0.001197,-0.010168,-0.026838,-0.048347,0.372044
56358,31,1,0.049180,23686,129,10,1,1,4.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
56359,31,0,0.049689,29370,158,10,0,0,1.0,0.0,...,0.114802,-0.013694,-0.019631,-0.025764,-0.024707,0.126377,0.038196,0.047655,0.237562,-0.079498
56360,31,0,0.049180,22609,398,10,1,0,4.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [136]:
id_list = x_val['id']
x_val = x_val.drop('id', axis = 1)
x_val

,expected_budget,business_unit,com_reg_ver_win_rate,customer_idx,lead_date,customer_type,customer_history,enterprise,historical_existing_cnt,id_strategic_ver,...,0,1,2,3,4,5,6,7,8,9
0,31,1,0.044643,19119,835,5,1,1,4.0,1.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,35,0,0.147059,35859,859,5,1,0,4.0,0.0,...,0.251493,0.009624,-0.075126,-0.104187,-0.037233,0.095518,0.027773,0.317199,-0.156316,-0.072153
2,1,0,0.461538,38658,874,5,1,1,4.0,0.0,...,0.342355,0.045087,-0.220965,-0.136603,-0.063918,0.142668,0.039530,-0.046058,-0.127773,0.219151
3,46,0,0.049180,29043,813,5,1,0,4.0,0.0,...,0.051706,0.004954,-0.049751,-0.002579,-0.002984,-0.013534,-0.006416,0.008228,-0.007432,0.014432
4,35,1,0.028926,42617,801,1,1,1,4.0,0.0,...,0.303776,-0.124053,0.004358,0.212029,0.028287,-0.131175,-0.033069,-0.041004,-0.136848,-0.131698
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21336,31,0,0.049180,31080,854,10,1,0,4.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
21337,7,0,0.049180,43632,813,10,0,1,271.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
21338,46,0,0.049180,39855,884,15,1,1,4.0,0.0,...,0.235611,0.031091,0.135517,0.039883,-0.026080,0.191438,0.057880,0.063255,0.349068,-0.074266
21339,31,0,0.049180,47466,900,5,0,0,16.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [137]:
# 범주형인 변수
cat_cols = np.where(x_train.dtypes != np.float64)[0] 
cat_cols

array([ 0,  1,  3,  4,  5,  6,  7, 10, 11, 13, 14, 17, 18, 19, 20, 21, 22,
       23, 24, 25, 29, 30, 31, 32, 33, 34])

In [138]:
# 연속형인 변수
ncat_cols = np.where(x_train.dtypes == np.float64)[0]
ncat_cols

array([ 2,  8,  9, 12, 15, 16, 26, 27, 28, 35, 36, 37, 38, 39, 40, 41, 42,
       43, 44])

In [139]:
# min-max scaling

nms = MinMaxScaler()

nms.fit(x_train.iloc[:, ncat_cols])
x_train.iloc[:, ncat_cols] = nms.transform(x_train.iloc[:, ncat_cols])
x_val.iloc[:, ncat_cols] = nms.transform(x_val.iloc[:, ncat_cols])

### modeling

In [143]:
def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, y_pred, labels=[True, False])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[True, False])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[True, False])

    print("오차행렬:\n", confusion)
    print("\n정확도: {:.4f}".format(accuracy))
    print("정밀도: {:.4f}".format(precision))
    print("재현율: {:.4f}".format(recall))
    print("F1: {:.4f}".format(F1))
    
#cat_clf = CatBoostClassifier(class_weights = class_weights, cat_features = cat_cols, verbose = 500, **params)
#cv_scores = cross_val_score(cat_clf, x_train, y_train, scoring = 'f1', cv = StratifiedKFold(n_splits = 7, shuffle = True))
#print(cv_scores.mean(), cv_scores.std())

In [179]:
predict = np.zeros((len(x_val), 5))
cat_clf = CatBoostClassifier(class_weights = class_weights, cat_features = cat_cols, verbose = 500)
cat_clf.fit(x_train, y_train)
pred = cat_clf.predict_proba(x_val)[:, 1]
predict[:, 0] = pred

Learning rate set to 0.057624
0:	learn: 0.5860927	total: 122ms	remaining: 2m 2s
500:	learn: 0.0938115	total: 41.9s	remaining: 41.7s
999:	learn: 0.0710926	total: 1m 23s	remaining: 0us


In [180]:
cat_params1 = {'iterations': 921, 'od_wait': 17, 'learning_rate': 0.17923424842472702, 'reg_lambda': 82.76322648574165, 'subsample': 0.587171260125725, 'random_strength': 28.37414982578848, 'depth': 9, 'min_data_in_leaf': 25, 'leaf_estimation_iterations': 5, 'bagging_temperature': 0.06361763250150829, 'colsample_bylevel': 0.6419394747181384}
cat_params2 = {'iterations': 1051, 'od_wait': 110, 'learning_rate': 0.24583120160214106, 'reg_lambda': 35.74576203788852, 'subsample': 0.6696979025262206, 'depth': 8, 'min_data_in_leaf': 8, 'bagging_temperature': 0.4566699465614833, 'colsample_bylevel': 0.770172714133851}
cat_params3 = {'iterations': 705, 'od_wait': 17, 'learning_rate': 0.17923424842472702, 'reg_lambda': 82.76322648574165, 'subsample': 0.587171260125725, 'random_strength': 28.37414982578848, 'depth': 9, 'min_data_in_leaf': 25, 'leaf_estimation_iterations': 5, 'bagging_temperature': 0.06361763250150829, 'colsample_bylevel': 0.6419394747181384}
cat_params4 = {'iterations': 1006, 'od_wait': 507, 'learning_rate': 0.06320554589891039, 'reg_lambda': 7.7075492078826535, 'subsample': 0.24445127904293296, 'random_strength': 12.664216911848179, 'depth': 7, 'min_data_in_leaf': 27, 'colsample_bylevel': 0.9908997952469817}

params = [cat_params1, cat_params2, cat_params3, cat_params4]

x_test = df_test.drop(['id', 'is_converted'], axis = 1)
x_test.iloc[:, ncat_cols] = nms.transform(x_test.iloc[:, ncat_cols])

#predict = np.zeros((len(x_test), len(params)))
for i in range(len(params)):
    cat_clf = CatBoostClassifier(class_weights = class_weights, cat_features = cat_cols, verbose = 500, **params[i])
    cat_clf.fit(x_train, y_train)
    pred = cat_clf.predict_proba(x_val)[:, 1]
    predict[:, i+1] = pred

0:	learn: 0.5472917	total: 115ms	remaining: 1m 46s
500:	learn: 0.0806415	total: 59.4s	remaining: 49.8s
920:	learn: 0.0667355	total: 1m 50s	remaining: 0us
0:	learn: 0.3833270	total: 145ms	remaining: 2m 32s
500:	learn: 0.0594992	total: 54.1s	remaining: 59.4s
1000:	learn: 0.0399256	total: 1m 49s	remaining: 5.46s
1050:	learn: 0.0384458	total: 1m 54s	remaining: 0us
0:	learn: 0.5472917	total: 108ms	remaining: 1m 16s
500:	learn: 0.0806415	total: 59.4s	remaining: 24.2s
704:	learn: 0.0703371	total: 1m 24s	remaining: 0us
0:	learn: 0.6570080	total: 112ms	remaining: 1m 52s
500:	learn: 0.0978401	total: 36.8s	remaining: 37.1s
1000:	learn: 0.0735902	total: 1m 16s	remaining: 381ms
1005:	learn: 0.0734613	total: 1m 16s	remaining: 0us


### predict

In [181]:
predict

array([[2.17728547e-02, 1.95296538e-02, 1.12236695e-02, 2.21656076e-02,
        1.89982524e-02],
       [6.34322727e-01, 4.99875835e-01, 1.63388066e-01, 5.32923412e-01,
        4.72366603e-01],
       [5.12533919e-01, 5.60200561e-01, 1.26846313e-01, 6.13380896e-01,
        5.83333838e-01],
       ...,
       [3.92635823e-02, 1.71584478e-01, 1.17651584e-02, 1.99780135e-01,
        7.88281845e-02],
       [2.15896993e-03, 9.88745447e-04, 3.57230027e-04, 1.20217821e-03,
        2.27366061e-03],
       [6.95614191e-04, 6.54404584e-04, 7.30290923e-05, 6.25209561e-04,
        1.53365120e-04]])

In [182]:
cat_pred = np.mean(predict, axis = 1) >= 0.5
sum(cat_pred)

1829

In [183]:
df_sub = pd.read_csv('submission.csv')
df_sub['is_converted'] = cat_pred.astype(int)
df_sub.to_csv('submission.csv', index = False)
df_sub['is_converted'].sum()

1829

In [187]:
np.mean(predict, axis = 1) * 100

array([1.87380076e+00, 4.60575329e+01, 4.79259105e+01, ...,
       1.00244308e+01, 1.39615684e-01, 4.40324510e-02])

In [200]:
weights = np.array([0.4, 0.25, 0.2, 0.1, 0.05])

# weighted average 계산
weighted_avg = np.sum(predict * weights, axis=1)

print(weighted_avg)

[0.01900276 0.48828633 0.46093775 ... 0.08487401 0.00141612 0.00052664]


In [201]:
result = np.where(weighted_avg >= 0.5, 1, 0)
sum(result)

1865

In [202]:
df_sub = pd.read_csv('submission.csv')
df_sub['is_converted'] = result.astype(int)
df_sub.to_csv('submission.csv', index = False)
df_sub['is_converted'].sum()

1865